In [8]:
# load the dataset
import numpy as np

data = np.load('../dataset/kmeans_data.npy')

In [9]:
# use manhattan distance as the distance metric for clustering procedures
def dist(x,centroids):
  out = [0]*centroids.shape[0]
  for i in range(centroids.shape[0]):
    out[i] = abs(x[0]-centroids[i][0]) + abs(x[1]-centroids[i][1])
  return out


# manhattan distance between two points
def dpts(x,y):
  return (abs(x[0]-y[0]) + abs(x[1]-y[1]))

k-means Clustering

In [10]:
# Main k-means Clustering Algorithm
def kmeans_algo(k):

  # choose k random points as initial centroids
  centroids = data[np.random.choice(data.shape[0],size=k,replace=False)]

  # set maximum iterations to 1000
  max = 1000

  # data pt labels (cluster IDs)
  labels = np.array([-1]*data.shape[0])

  while(max>0):
    
    # assign data point to clusters based on proximity
    for j in range(data.shape[0]):
      labels[j] = np.argmin(dist(data[j],centroids),axis=0)
    
    # reassign cluster-centroids after assigning pts to clusters
    new_centroids = np.copy(centroids)
    for j in range(k):
      xsum = 0
      ysum = 0
      n = 0
      for x in range(data.shape[0]):
        if(labels[x]==j):
          xsum += data[x][0]
          ysum += data[x][1]
          n += 1
      new_centroids[j] = [xsum/float(n),ysum/float(n)]
    
    # check if shift in centroids is less than 0.001
    # if yes, then return the current labels and centroids
    # else continue until convergence or max iterations
    for i in range(len(centroids)):
      if(abs(centroids[i][0]-new_centroids[i][0]) + abs(centroids[i][1]-new_centroids[i][1]) > 0.00002):
        centroids = np.copy(new_centroids)
        break
      if(i==len(centroids)-1):
        return labels, new_centroids

    max -= 1

  return labels, centroids

In [11]:
k_vals = range(4,10)
avg_si_scores = []

for k in k_vals:
  
  # apply k-means clustering for current k value
  l,c = kmeans_algo(k)

  # Silhouette score for current k value:

  # calculate (manhattan) distance between each data pt and each centroid
  d = (abs(data - c[l, :])).sum(axis=1)

  si_score = []
  for i in range(data.shape[0]):
    # define the cluster with cluster ID i
    ci = d[l==l[i]]

    # calculate a(i) as the intra cluster distance
    ai = 0
    if(ci.shape[0]>1):
      # exclude the current data pt from the summation in a(i)
      ai = np.mean(ci[ci!=d[i]],axis=0)

    bi = np.inf
    for j in range(0,k):
      if(j==l[i]):
        continue
      
      # calculate distance between cluster J and current data point
      # and update b(i) as the minimum value of such a distance for a
      # cluster J != I
      s = 0
      m = 0
      for t in range(data.shape[0]):
        if(l[t]!=j):
          continue
        x = data[t]
        s += abs(x[0]-data[i][0]) + abs(x[1]-data[i][1])
        m += 1
      s /= float(m)
      bi = min(bi,s)
    
    # store the s(i) score for the current data point
    si_score.append((bi-ai)/max(ai,bi))
  
  # calculate and store the average s(i) score for current k value
  avg_si_score = np.mean(si_score)
  avg_si_scores.append(avg_si_score)

In [12]:
# print all k-values with corresponding silhouette scores
# and the optimal k-value
for i in range(len(avg_si_scores)):
  print("k-value: ",end="")
  print(k_vals[i])
  print("Avg silhouette score for k=",end="")
  print(k_vals[i],end="")
  print(": ",end="")
  print(avg_si_scores[i])
  print()

print("Optimal k-value: ",end="")
print(k_vals[np.argmax(avg_si_scores)])

k-value: 4
Avg silhouette score for k=4: 0.7845729441853112

k-value: 5
Avg silhouette score for k=5: 0.8142079920084564

k-value: 6
Avg silhouette score for k=6: 0.7523566358147558

k-value: 7
Avg silhouette score for k=7: 0.7464248904764451

k-value: 8
Avg silhouette score for k=8: 0.6939150984247473

k-value: 9
Avg silhouette score for k=9: 0.6071469796660225

Optimal k-value: 5


Fuzzy c-means

In [13]:
# define objective function of fuzzy c-means
def J(U,V,m):
  out = 0
  for i in range(U.shape[0]):
    for j in range(V.shape[0]):
      out += ((U[i][j])**m)*((data[i][0]-V[j][0])**2 + (data[i][1]-V[j][1])**2)
  return out


# Main Fuzzy c-means Clustering Algorithm
def fuzzy_algo(c,m,beta=0.3):

  # define n1 as the number of samples
  n1 = data.shape[0]
  
  # choose k random points as initial centroids
  V = data[np.random.choice(n1,size=c,replace=False)]

  # set maximum iterations to 500
  max = 500

  # define membership matrix
  U = np.random.rand(n1,c)
  # normalise matrix such that entries of each row sum up to 1
  U /= np.sum(U,axis=1)[:,np.newaxis]

  while(max>0):
    # if objective function takes value below threshold, return U,V
    if(J(U,V,m)<beta):
      return U,V
    
    for i in range(n1):
      for j in range(c):
        
        # find the membership of each data point w.r.t each cluster
        s = 0
        for k in range(c):
          s += (float(dpts(data[i],V[j]))/(dpts(data[i],V[k])))**(2.0/m - 1)
        
        # assign membership to point i wrt cluster j
        U[i][j] = 1.0/s
    
    # find fuzzy centroids
    for j in range(c):
      snumx = 0
      snumy = 0
      sden = 0
      for i in range(n1):
        snumx += ((U[i][j])**m)*data[i][0]
        snumy += ((U[i][j])**m)*data[i][1]
        sden += (U[i][j])**m
      
      # assign new centroid value to fuzzy centroids
      V[j] = np.array([float(snumx)/sden,float(snumy)/sden])
    
    max -= 1

  return U,V

In [14]:
c = k_vals[np.argmax(avg_si_scores)]
m = 2

# call fuzzy c-means for c = optimal k-value from k-means, and m = 2
U1,V1 = fuzzy_algo(c,m)

print("For c = ",end = "")
print(c,end="")
print(", for fuzzy c-means clustering, objective function takes value: ")
print(J(U1,V1,m))

/var/folders/fx/m8jghh2j0xn7xvzxjj0rz1sr0000gn/T/ipykernel_47583/3994027339.py:38: RuntimeWarning: invalid value encountered in double_scalars
  s += (float(dpts(data[i],V[j]))/(dpts(data[i],V[k])))**(2.0/m - 1)
/var/folders/fx/m8jghh2j0xn7xvzxjj0rz1sr0000gn/T/ipykernel_47583/3994027339.py:38: RuntimeWarning: divide by zero encountered in double_scalars
  s += (float(dpts(data[i],V[j]))/(dpts(data[i],V[k])))**(2.0/m - 1)


For c = 5, for fuzzy c-means clustering, objective function takes value: 
18644.69828228233
